# MeshOps
Operate on the meshes (correct nodes) and then convert to volumes (binarize)

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import os
from headctools.preprocessing import utils

path_imgs = os.path.abspath('../datasets/cq500mesh')
path_test = os.path.join(path_imgs, 'test')
path_sim = os.path.join(path_test, 'sim_defects')
path_rec_sim = os.path.join(path_sim, 'PCA_im2mesh')

/home/fmatzkin/Code/envs/anaconda3/envs/msr/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Convert the input volumes to meshes 

..so I can align the output meshes later

In [7]:
utils.nii_to_stl_marching_cubes(path_sim, path_sim, False, '_sim.nii.gz')
paths_inp_mshs = [os.path.join(path_sim, f) for f in os.listdir(path_sim)
                  if f.endswith('_sim.stl')]

    File /media/fmatzkin/data/franco/Code/datasets/cq500mesh/test/sim_defects/CQ500-CT-392_CT PLAIN THIN0_sim.stl already exists. Skipping...
    File /media/fmatzkin/data/franco/Code/datasets/cq500mesh/test/sim_defects/CQ500-CT-49_CT PRE CONTRAST THIN0_sim.stl already exists. Skipping...
    File /media/fmatzkin/data/franco/Code/datasets/cq500mesh/test/sim_defects/CQ500-CT-287_CT Thin Plain0_sim.stl already exists. Skipping...
    File /media/fmatzkin/data/franco/Code/datasets/cq500mesh/test/sim_defects/CQ500-CT-195_CT PRE CONTRAST THIN0_sim.stl already exists. Skipping...
    File /media/fmatzkin/data/franco/Code/datasets/cq500mesh/test/sim_defects/CQ500-CT-373_CT Thin Plain0_sim.stl already exists. Skipping...
    File /media/fmatzkin/data/franco/Code/datasets/cq500mesh/test/sim_defects/CQ500-CT-444_CT Thin Plain0_sim.stl already exists. Skipping...
    File /media/fmatzkin/data/franco/Code/datasets/cq500mesh/test/sim_defects/CQ500-CT-103_CT Thin Plain0_sim.stl already exists. Skipp

## Align the predicted mesh using ICP

In [37]:
import vedo
import SimpleITK as sitk
from headrecbaselines.utils.VedoMorpher3 import fit_mesh_dmap

corr_meshes_path = []  # TODO Crear dsp usando listdir y endswith
# Simulated meshes paths
meshes_paths = [os.path.join(path_rec_sim, f)
                for f in os.listdir(path_rec_sim) if f.endswith('_sim.stl')]

n_mshs = len(meshes_paths)
for i, otp_msh_pth in enumerate(meshes_paths):
    msh_fn = os.path.split(otp_msh_pth)[1]
    otp_corr_msh_pth = os.path.join(path_rec_sim, 
                                    msh_fn.replace('.stl', '_corr.stl'))
    inp_msh_pth = os.path.join(path_sim, msh_fn)
    dmap_pth = os.path.join(path_sim, 
                            msh_fn.replace('.stl', '_distmap.nii.gz'))
    
    print(f'[{i+1}/{n_mshs}] Aligning {msh_fn} to its input..')
    
    if not os.path.exists(inp_msh_pth):
        print(f'Input not found for {msh_fn}')
        continue
    otp_msh = vedo.Mesh(otp_msh_pth)
    inp_msh = vedo.Mesh(inp_msh_pth)
    dmp_vol = sitk.ReadImage(dmap_pth)
    
    pm = {
        'fit_only_common': True,  # only fit common points based on distance map
        'save_path': otp_corr_msh_pth,  # path to save the morphed mesh
        'dm_threshold': 0.5,  # distance map threshold
        'icp': True,  # Use ICP before to fit the meshes
        'closest_point_fit': False,  # Use closest point fit (using normals)
    }
    
    mrph_msh = fit_mesh_dmap(otp_msh, inp_msh, dmp_vol, pm)  # Morphed mesh

[1/36] Aligning CQ500-CT-47_CT PRE CONTRAST THIN0_sim.stl to its input..
Morphed mesh saved to /media/fmatzkin/data/franco/Code/datasets/cq500mesh/test/sim_defects/PCA_im2mesh/CQ500-CT-47_CT PRE CONTRAST THIN0_sim_corr.stl
[2/36] Aligning CQ500-CT-49_CT PRE CONTRAST THIN0_sim.stl to its input..
Morphed mesh saved to /media/fmatzkin/data/franco/Code/datasets/cq500mesh/test/sim_defects/PCA_im2mesh/CQ500-CT-49_CT PRE CONTRAST THIN0_sim_corr.stl
[3/36] Aligning CQ500-CT-462_CT Thin Plain0_sim.stl to its input..
Morphed mesh saved to /media/fmatzkin/data/franco/Code/datasets/cq500mesh/test/sim_defects/PCA_im2mesh/CQ500-CT-462_CT Thin Plain0_sim_corr.stl
[4/36] Aligning CQ500-CT-356_CT PLAIN THIN0_sim.stl to its input..
Morphed mesh saved to /media/fmatzkin/data/franco/Code/datasets/cq500mesh/test/sim_defects/PCA_im2mesh/CQ500-CT-356_CT PLAIN THIN0_sim_corr.stl
[5/36] Aligning CQ500-CT-384_CT PLAIN THIN0_sim.stl to its input..
Morphed mesh saved to /media/fmatzkin/data/franco/Code/datasets/c

IndexError: index 233 is outside the extent for dimension 2 with size 233

In [ ]:
import vedo
inpmesh = vedo.Mesh('/media/fmatzkin/data/franco/Code/datasets/cq500mesh/test/sim_defects/CQ500-CT-80_CT 0.625mm0_sim.stl')
predmesh = vedo.Mesh('/media/fmatzkin/data/franco/Code/datasets/cq500mesh/test/sim_defects/PCA_im2mesh/CQ500-CT-80_CT 0.625mm0_sim.stl')


In [ ]:
predmesh.show()

In [6]:
# Draw an arrow for each normal of the inpmesh
arrs = []
for i, pt in enumerate(inpmesh.points()):
    arrs.append([pt, pt + inpmesh.normals()[i] * 10])
v_arrs = vedo.Arrows(arrs)
vedo.show(inpmesh, v_arrs).close()

KeyboardInterrupt: 

In [7]:
# Draw an arrow for each normal of the prediction meshq
arrs = []
for i, pt in enumerate(predmesh.points()):
    arrs.append([pt, pt + predmesh.normals()[i] * 10])
v_arrs = vedo.Arrows(arrs)
vedo.show(predmesh, v_arrs).close()

: 

: 

In [ ]:
import trimesh
trimesh.load_mesh()

## Convert meshes to volumes
Using as a reference the image CQ500-CT-0_CT PLAIN THIN.nii.gz of the training split (for dims/metadata), binarize the meshes and save them as volumes (.nii).

In [ ]:
from headrecbaselines.utils.utils import mesh2vol

ref_im = os.path.join(path_imgs, 'CQ500-CT-0_CT PLAIN THIN.nii.gz')

for mesh in corr_meshes_path:
    mesh2vol(mesh, ref_im)

/home/fmatzkin/Code/envs/anaconda3/envs/msr/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Binarizing /media/fmatzkin/data/franco/Code/datasets/cq500mesh/test/sim_defects/PCA_im2mesh/CQ500-CT-47_CT PRE CONTRAST THIN0_sim.stl..done.
Binarizing /media/fmatzkin/data/franco/Code/datasets/cq500mesh/test/sim_defects/PCA_im2mesh/CQ500-CT-49_CT PRE CONTRAST THIN0_sim.stl..  file /media/fmatzkin/data/franco/Code/datasets/cq500mesh/test/sim_defects/PCA_im2mesh/CQ500-CT-49_CT PRE CONTRAST THIN0_sim_binMesh.nii.gz already exists.
Binarizing /media/fmatzkin/data/franco/Code/datasets/cq500mesh/test/sim_defects/PCA_im2mesh/CQ500-CT-462_CT Thin Plain0_sim.stl..  file /media/fmatzkin/data/franco/Code/datasets/cq500mesh/test/sim_defects/PCA_im2mesh/CQ500-CT-462_CT Thin Plain0_sim_binMesh.nii.gz already exists.
Binarizing /media/fmatzkin/data/franco/Code/datasets/cq500mesh/test/sim_defects/PCA_im2mesh/CQ500-CT-356_CT PLAIN THIN0_sim.stl..  file /media/fmatzkin/data/franco/Code/datasets/cq500mesh/test/sim_defects/PCA_im2mesh/CQ500-CT-356_CT PLAIN THIN0_sim_binMesh.nii.gz already exists.
Binariz